# Agenda for Jupyter
<img src="https://jupyter.org/assets/main-logo.svg" style="float:right;height:240pt"/>

1. Configure Your Settings
1. Connect to Snowflake
1. Query Snowflake Data
1. Parametrizing Queries
1. Stat Plan Python Routines
1. Manipulating Pandas DataFrames


## Configure Your Settings

Navigate to **📁 / aeconf / aeuser.txt**

**Lines 18-22 are the same for everyone:**

```
## Snowflake
## ----------------
SFAUTHENTICATOR='https://verisk.okta.com'
SFACCOUNT='verisk.us-east-1.privatelink'
SFWAREHOUSE='IPO_ANALYTICS_WH'
```

```
SFDATABASE='YOUR_GROUP_DATABASE'
SFSCHEMA='YOUR_SCHEMA_CREATED_EARLIER'
SFUSER='YOUR-i-NUMBER'
SFROLE='YOUR_ROLE'
```

## Connect to Snowflake

In [ ]:
# System packages imports
import os
import pandas as pd
from getpass import getpass

In [ ]:
# Import our in-house package to connect to Snowflake
import sfcommon.sfdb as sf

In [ ]:
# Prompt your password since we did not save it in aeconf.txt
# Reference: https://docs.python.org/3.7/library/getpass.html
os.environ['SFPASSWORD'] = getpass(prompt='Password: ', stream=None)

Password:  ············


In [ ]:
# This will read aeconf.txt settings that we configured earlier,
# and use those settings to initailize a connection to Snowflake database
# Reference: https://docs.snowflake.com/en/user-guide/python-connector.html
sf_conn = sf.get_connect()

In [ ]:
# This will enable you to perform sql_magic in your notebook. 
# sql_magic is Jupyter magic for writing SQL to interact with relational databases. 
# Query results are saved directly to a Pandas dataframe.
# Reference: https://github.com/pivotal-legacy/sql_magic
%reload_ext sql_magic
%config SQL.conn_name = 'sf_conn' # we defined sf_conn in the cell above

## Query Snowflake Data

How many records are in our dataset?

In [ ]:
%%read_sql
SELECT count(1) as cnt
FROM IPO_DEAL_DB.Q1_2020_TRAINING.BLDG_CONT_PREM_SUBSET

Query started at 10:56:16 AM EDT; Query executed in 0.00 m

,CNT
0,1263221


What's the aggregate premium amount and policy count per (constr_mtrl, st, inc_yr) grouping?

A unique policy is assumed to be defined as having distinct (tl_grp, co, prem_rec_id, inc_yr) values

In [ ]:
%%read_sql -d df_input_selections
SELECT 
  constr_mtrl,
  st,
  inc_yr,
  SUM(prem_amt) AS prem_amt, -- aggregate premium amounts
  SUM(expo) AS expo, -- aggregate exposure
  COUNT(DISTINCT(tl_grp||co||prem_rec_id||inc_yr)) AS pol_cnt -- count of unique policies
FROM 
  IPO_DEAL_DB.Q1_2020_TRAINING.BLDG_CONT_PREM_SUBSET
WHERE
  (inc_yr::INTEGER BETWEEN 2014 AND 2018) AND -- filter to records with inception year between 2014 and 2018
  closeout_date <= 20192 AND -- filter to closeout as of 20192
  bldg_cont_cov_id = 'bldg' AND -- filter to 'bldg' coverage records
  liab_prop_cov_id IN ('prop', 'comb') AND -- filter to 'prop' and 'comb' coverage records
  terror_cov_cd NOT IN ('7', '8') AND -- exclude certain terror coverages '7' and '8'
  asgnd_sev IN ('0', '1') -- only severity of 0 or 1 has data quality worth processing
GROUP BY 
  constr_mtrl, st, inc_yr
ORDER BY 
  constr_mtrl, st, inc_yr;

Query started at 11:01:17 AM EDT; Query executed in 0.02 m

In [ ]:
df_input_selections  # contains results of our last query

,CONSTR_MTRL,ST,INC_YR,PREM_AMT,EXPO,POL_CNT
0,fire resistive,19,2014,399727.0,4105155.0,107
1,fire resistive,19,2015,495395.0,4991156.0,109
2,fire resistive,19,2016,658596.0,6767995.0,106
3,fire resistive,19,2017,644850.0,6599928.0,107
4,fire resistive,19,2018,688464.0,6218659.0,107
5,frame,19,2014,5426771.0,28841413.0,2063
6,frame,19,2015,6124989.0,34580610.0,2070
7,frame,19,2016,7168403.0,41661453.0,2061
8,frame,19,2017,7908189.0,48697337.0,2030
9,frame,19,2018,8495693.0,52784555.0,2022


In [ ]:
type(df_input_selections)

pandas.core.frame.DataFrame

### Quick Note on Magic Commands: `%` and `%%`

In [ ]:
%%read_sql
-- Description of my code
/*
Must use SQL-style comments because %%read_sql tells Jupyter to treat this whole cell as SQL

nothing is allowed above %%read_sql

and no Python is allowed in this cell
*/

In [ ]:
print('This passes!')

%read_sql SELECT COUNT(1) FROM IPO_DEAL_DB.Q1_2020_TRAINING.BLDG_CONT_PREM_SUBSET;

Query started at 11:07:10 AM EDT; Query executed in 0.01 m

,COUNT(1)
0,1263221


## Parametrizing Queries: Using Python Variables in SQL
Previous code requires you to manually update SQL to run with new filter conditions, but we can do better.

In [ ]:
# Let's define some varaibles in Python
input_relation = 'IPO_DEAL_DB.Q1_2020_TRAINING.BLDG_CONT_PREM_SUBSET'
my_table_name = 'input_selections_i26250'
inc_start_yr = 2014
inc_end_yr = 2018
closeout_date = 20192
bldg_cont_cov_id = 'bldg'

In [ ]:
%%read_sql
CREATE OR REPLACE TEMP TABLE {my_table_name} AS (
    SELECT 
      inc_yr, st, constr_mtrl, 
      SUM(prem_amt) AS prem_amt,
      SUM(expo) AS expo,
      COUNT(DISTINCT(tl_grp||co||prem_rec_id||inc_yr)) AS pol_cnt
    FROM 
      {input_relation}
    WHERE
      inc_yr::INTEGER BETWEEN {inc_start_yr}  AND {inc_end_yr} AND
      closeout_date <= {closeout_date} AND
      lower(bldg_cont_cov_id) = '{bldg_cont_cov_id}' AND
      lower(liab_prop_cov_id) IN ('prop', 'comb') AND
      terror_cov_cd NOT IN ('7', '8') AND
      asgnd_sev IN ('0', '1')
    GROUP BY 
      constr_mtrl, st, inc_yr
    ORDER BY 
      constr_mtrl, st, inc_yr
);

Query started at 11:11:11 AM EDT; Query executed in 0.02 m

,status
0,Table INPUT_SELECTIONS_I26250 successfully cre...


In [ ]:
%%read_sql

SELECT * FROM {my_table_name};

Query started at 11:12:10 AM EDT; Query executed in 0.02 m

,INC_YR,ST,CONSTR_MTRL,PREM_AMT,EXPO,POL_CNT
0,2018,19,joisted masonry,7462963.0,40569766.0,2444
1,2014,19,masonry non-combustible,4015886.0,31583871.0,1379
2,2016,19,masonry non-combustible,4399870.0,34878065.0,1370
3,2014,19,non-combustible,1356293.0,8619992.0,573
4,2014,19,fire resistive,399727.0,4105155.0,107
5,2015,19,fire resistive,495395.0,4991156.0,109
6,2016,19,fire resistive,658596.0,6767995.0,106
7,2018,19,frame,8495693.0,52784555.0,2022
8,2017,19,modified firse resistive,83263.0,524641.0,15
9,2017,19,joisted masonry,7092819.0,37285829.0,2503


In [ ]:
%read_sql SELECT COUNT(1) as cnt FROM {my_table_name};

Query started at 11:16:59 AM EDT; Query executed in 0.04 m

,CNT
0,30


## Stat Plan Python Routines

<a href="https://epm.verisk.com/confluence/pages/viewpage.action?pageId=177871408">Actuarial Routines Documentation</a>


### Quick Note on Dictionaries

In [ ]:
my_dict = {'fname': 'Pranav', 'lname': 'Singh', 'friends': []}
print(type(my_dict)) # what type of python object is 'd1'?
print(my_dict) # print the entire dictionary

<class 'dict'>
{'fname': 'Pranav', 'lname': 'Singh', 'friends': []}


In [ ]:
# What are the dictionary keys?
my_dict.keys()

dict_keys(['fname', 'lname', 'friends'])

In [ ]:
# What are the dictionary values?
my_dict.values()

dict_values(['Pranav', 'Singh', []])

In [ ]:
my_dict['fname']

'Pranav'

In [ ]:
# Print each (key, value) item in the dictionary, line by line:
for (key, value) in my_dict.items():
    print('key:', key, 'value:', value)

key: fname value: Pranav
key: lname value: Singh
key: friends value: []


### Adjust Exposures by Term

In [ ]:
# Import the python class for earnings from the statisitcal plan common library spcommom
from spcommon.adj_expo_by_term import AdjExpoByTerm

print(AdjExpoByTerm.__doc__)  # show documentation for this routine


    Objective :
        Procedure adj_expo_by_term takes as an input any premium relation and returns a relation
        with adjusted exposure and a signed term value.
        Depending on the exposure base reported an adjustment might be necessary to account for the term of the policy.
        A common case is amount of insurance. If the amount of insurance is 500K and the policy is for 1 year this is
        a significantly different exposure than if it was 500K for 1.5 years.
        The adjustment is based on both the line of business and the exposure base and is only applied when appropriate.

        For more information please look below to get documentation link .

    adj_expo_by_term Usage Example/Syntax :
        conn = sfdb.get_connect()   # User needs to create connection. Mandatory for further run.

        input_param={'input_relation':         'ae_qa_db.ipascommon.input_adjust_expo_by_term_genliab',
                     'lob':                    'genliab',
           

In [ ]:
input_param = {
    'input_relation': input_relation,
    'lob': 'other',
    'signed_term_indicator': 'y'
}

In [ ]:
input_param

{'input_relation': 'IPO_DEAL_DB.Q1_2020_TRAINING.BLDG_CONT_PREM_SUBSET',
 'lob': 'other',
 'signed_term_indicator': 'y'}

In [ ]:
# initialize the AdjExpoByTerm procedure
adj_expo_instance = AdjExpoByTerm(sf_conn, input_param)

In [ ]:
# get documentation on the AdjExpoByTerm procedure
adj_expo_instance.get_documentation()

https://epm.verisk.com/confluence/display/IPASAENV/Proc_adjust_expo_by_term


In [ ]:
dir(adj_expo_instance)

['__abstractmethods__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'all_params',
 'append_or_run_query',
 'check_mandatory_params',
 'check_relation_for_join_keys',
 'conn',
 'convert_to_dbschrel',
 'get_db_params',
 'get_documentation',
 'get_rel_suffix',
 'link',
 'param',
 'process_sql',
 'query',
 'set_optional_params',
 'set_output_relation_type',
 'sfutil',
 'show_sql',
 'tech_params',
 'validate_input_params']

In [ ]:
# shows the SQL underlying the procedure, without actually executing it
print(adj_expo_instance.show_sql())


                DROP TABLE IF EXISTS IPO_DEAL_DB.Q1_2020_TRAINING.temp_adj_expo;
                CREATE TEMP TABLE IPO_DEAL_DB.Q1_2020_TRAINING.temp_adj_expo AS
                SELECT
                    *,
                    (expo * term)::NUMERIC(12,4) as expo_adjusted
                    ,
                     CASE WHEN (sign(prem_amt) = 0 )
                             THEN 0
                          WHEN (sign(prem_amt) = 1 )
                             THEN term::NUMERIC(12,4)
                          WHEN (sign(prem_amt) = -1 )
                             THEN (term * -1)::NUMERIC(12,4)
                     END AS signed_term
                  
                FROM (SELECT
                          *,
                          CASE WHEN exp_yr is NULL OR trim(exp_yr) = ''
                                   THEN (.5/12)
                               WHEN 
                                NOT REGEXP_LIKE(eff_yr , '^19[789][0-9]|20[012345][0-9]$') OR
                         

In [ ]:
# executes the SQL and save the result as a variable in python
adj_expo_output = adj_expo_instance.process_sql()

In [ ]:
# print the table name on Snowflake containing the results
adj_expo_output

'IPO_DEAL_DB.Q1_2020_TRAINING.adj_expo_by_term_i26250_20200406_112752'

In [ ]:
%%read_sql output_2

SELECT 
  constr_mtrl,
  st,
  inc_yr,
  SUM(expo) as expo, -- aggregate exposure
  SUM(expo_adjusted) as expo_adjusted, -- aggregate adjusted exposure
  SUM(prem_amt) AS prem_amt, -- aggregate premium amounts
  COUNT(DISTINCT(tl_grp||co||prem_rec_id||inc_yr)) AS pol_cnt -- unique policies
FROM 
  {adj_expo_output}
WHERE
  (inc_yr::INTEGER BETWEEN {inc_start_yr} AND {inc_end_yr}) AND -- filter to records with inception year between 2014 and 2018
  closeout_date <= {closeout_date} AND -- filter to closeout as of 20192
  bldg_cont_cov_id = 'bldg' AND -- filter to 'bldg' coverage records
  liab_prop_cov_id IN ('prop', 'comb') AND -- filter to 'prop' and 'comb' coverage records
  terror_cov_cd NOT IN ('7', '8') AND -- exclude terror coverages '7' and '8'
  asgnd_sev IN ('0', '1') -- only severity of 0 or 1 has data quality worth processing
GROUP BY 
  constr_mtrl, st, inc_yr
ORDER BY 
  constr_mtrl, st, inc_yr;

Query started at 11:29:24 AM EDT; Query executed in 0.03 m

,CONSTR_MTRL,ST,INC_YR,EXPO,EXPO_ADJUSTED,PREM_AMT,POL_CNT
0,fire resistive,19,2014,4105155.0,4.163405e+06,399727.0,107
1,fire resistive,19,2015,4991156.0,5.151379e+06,495395.0,109
2,fire resistive,19,2016,6767995.0,6.838008e+06,658596.0,106
3,fire resistive,19,2017,6599928.0,6.658982e+06,644850.0,107
4,fire resistive,19,2018,6218659.0,6.748572e+06,688464.0,107
5,frame,19,2014,28841413.0,2.879528e+07,5426771.0,2063
6,frame,19,2015,34580610.0,3.464491e+07,6124989.0,2070
7,frame,19,2016,41661453.0,4.278891e+07,7168403.0,2061
8,frame,19,2017,48697337.0,4.918506e+07,7908189.0,2030
9,frame,19,2018,52784555.0,5.363458e+07,8495693.0,2022


## Manipulating Pandas DataFrames

In [ ]:
# Let's get some basic info about our dataframe
print(type(output_2))
output_2.info()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
CONSTR_MTRL      30 non-null object
ST               30 non-null object
INC_YR           30 non-null object
EXPO             30 non-null float64
EXPO_ADJUSTED    30 non-null float64
PREM_AMT         30 non-null float64
POL_CNT          30 non-null int64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.8+ KB


In [ ]:
# Let's do some standard calculations on all our numerical columns
output_2.describe()

,EXPO,EXPO_ADJUSTED,PREM_AMT,POL_CNT
count,3.000000e+01,3.000000e+01,3.000000e+01,30.000000
mean,2.117164e+07,2.158948e+07,3.396813e+06,1109.433333
std,1.697442e+07,1.727242e+07,2.946261e+06,980.359423
min,2.676070e+05,2.776070e+05,3.691800e+04,13.000000
25%,6.313976e+06,6.681380e+06,6.482865e+05,107.000000
50%,2.048012e+07,2.067764e+07,2.818824e+06,967.500000
75%,3.480370e+07,3.516291e+07,6.263651e+06,2053.250000
max,5.278456e+07,5.363458e+07,8.495693e+06,2689.000000


In [ ]:
# Let's select the first 3 columns of the first 13 rows
output_2.iloc[0:10, 0:3]

,CONSTR_MTRL,ST,INC_YR
0,fire resistive,19,2014
1,fire resistive,19,2015
2,fire resistive,19,2016
3,fire resistive,19,2017
4,fire resistive,19,2018
5,frame,19,2014
6,frame,19,2015
7,frame,19,2016
8,frame,19,2017
9,frame,19,2018


In [ ]:
# Let's select the first 10 records, and limit our output to just 2 columns: CONSTR_MTRL, EXPO_ADJUSTED
output_2.loc[0:9, ['CONSTR_MTRL', 'EXPO_ADJUSTED']]

,CONSTR_MTRL,EXPO_ADJUSTED
0,fire resistive,4.163405e+06
1,fire resistive,5.151379e+06
2,fire resistive,6.838008e+06
3,fire resistive,6.658982e+06
4,fire resistive,6.748572e+06
5,frame,2.879528e+07
6,frame,3.464491e+07
7,frame,4.278891e+07
8,frame,4.918506e+07
9,frame,5.363458e+07


In [ ]:
# Let's select records 26-30, and limit our output to all columns between CONSTR_MTRL and EXPO_ADJUSTED
output_2.loc[25:29, 'CONSTR_MTRL':'EXPO_ADJUSTED']

,CONSTR_MTRL,ST,INC_YR,EXPO,EXPO_ADJUSTED
25,non-combustible,19,2014,8619992.0,8.791122e+06
26,non-combustible,19,2015,8696385.0,9.181807e+06
27,non-combustible,19,2016,9553058.0,9.842325e+06
28,non-combustible,19,2017,11307510.0,1.151356e+07
29,non-combustible,19,2018,12360553.0,1.256000e+07


In [ ]:
# Which records have a policy count greater than 2000?
output_2.loc[(output_2['POL_CNT'] > 2000)]

,CONSTR_MTRL,ST,INC_YR,EXPO,EXPO_ADJUSTED,PREM_AMT,POL_CNT
5,frame,19,2014,28841413.0,2.879528e+07,5426771.0,2063
6,frame,19,2015,34580610.0,3.464491e+07,6124989.0,2070
7,frame,19,2016,41661453.0,4.278891e+07,7168403.0,2061
8,frame,19,2017,48697337.0,4.918506e+07,7908189.0,2030
9,frame,19,2018,52784555.0,5.363458e+07,8495693.0,2022
10,joisted masonry,19,2014,28599688.0,2.915586e+07,6309872.0,2689
11,joisted masonry,19,2015,29872971.0,3.071714e+07,6599896.0,2622
12,joisted masonry,19,2016,33589382.0,3.443954e+07,6871030.0,2533
13,joisted masonry,19,2017,37285829.0,3.840985e+07,7092819.0,2503
14,joisted masonry,19,2018,40569766.0,4.168317e+07,7462963.0,2444


In [ ]:
# Display our data sorted by premium amount in ascending order
output_2.loc[(output_2['POL_CNT'] > 2000)].sort_values(['INC_YR','PREM_AMT'], ascending=True)

,CONSTR_MTRL,ST,INC_YR,EXPO,EXPO_ADJUSTED,PREM_AMT,POL_CNT
5,frame,19,2014,28841413.0,2.879528e+07,5426771.0,2063
10,joisted masonry,19,2014,28599688.0,2.915586e+07,6309872.0,2689
6,frame,19,2015,34580610.0,3.464491e+07,6124989.0,2070
11,joisted masonry,19,2015,29872971.0,3.071714e+07,6599896.0,2622
12,joisted masonry,19,2016,33589382.0,3.443954e+07,6871030.0,2533
7,frame,19,2016,41661453.0,4.278891e+07,7168403.0,2061
13,joisted masonry,19,2017,37285829.0,3.840985e+07,7092819.0,2503
8,frame,19,2017,48697337.0,4.918506e+07,7908189.0,2030
14,joisted masonry,19,2018,40569766.0,4.168317e+07,7462963.0,2444
9,frame,19,2018,52784555.0,5.363458e+07,8495693.0,2022


In [ ]:
# Pivot our data: 
# for each state and for each inception year, 
# what is the aggregate premium amount, adjusted exposure, and policy count?
pivot1 = pd.pivot_table(output_2,
                        values=['PREM_AMT', 'EXPO_ADJUSTED', 'POL_CNT'],
                        index=['ST', 'INC_YR'],
                        aggfunc=sum,
                        margins = True)
pivot1

EXPO_ADJUSTED  POL_CNT     PREM_AMT
ST  INC_YR                                     
19  2014     1.033940e+08     6825   17545467.0
    2015     1.130717e+08     6731   18913690.0
    2016     1.296582e+08     6620   20571689.0
    2017     1.452190e+08     6606   21817236.0
    2018     1.563416e+08     6501   23056311.0
All          6.476845e+08    33283  101904393.0

In [ ]:
# Pivot our data: 
# for each state and for each inception year, 
# what is the aggregate premium amount, adjusted exposure, and policy count per construction material type?
pivot2 = pd.pivot_table(output_2,
                        values=['PREM_AMT', 'EXPO_ADJUSTED', 'POL_CNT'],
                        index=['ST', 'INC_YR'],
                        aggfunc=sum,
                        columns=['CONSTR_MTRL'],
                        margins = True)
pivot2

EXPO_ADJUSTED                                \
CONSTR_MTRL fire resistive         frame joisted masonry   
ST  INC_YR                                                 
19  2014      4.163405e+06  2.879528e+07    2.915586e+07   
    2015      5.151379e+06  3.464491e+07    3.071714e+07   
    2016      6.838008e+06  4.278891e+07    3.443954e+07   
    2017      6.658982e+06  4.918506e+07    3.840985e+07   
    2018      6.748572e+06  5.363458e+07    4.168317e+07   
All           2.956035e+07  2.090487e+08    1.744056e+08   

                                                                              \
CONSTR_MTRL masonry non-combustible modified firse resistive non-combustible   
ST  INC_YR                                                                     
19  2014               3.221071e+07                277607.00    8.791122e+06   
    2015               3.309234e+07                284136.00    9.181807e+06   
    2016               3.533558e+07                413889.00    9.842325e+06   
    2017               3.892692e+07                524641.00    1.151356e+07   
    2018               4.072915e+07                986067.25    1.256000e+07   
All                    1.802947e+08               2486340.25    5.188882e+07   

                                 POL_CNT                         ...  \
CONSTR_MTRL           All fire resistive  frame joisted masonry  ...   
ST  INC_YR                                                       ...   
19  2014     1.033940e+08            107   2063            2689  ...   
    2015     1.130717e+08            109   2070            2622  ...   
    2016     1.296582e+08            106   2061            2533  ...   
    2017     1.452190e+08            107   2030            2503  ...   
    2018     1.563416e+08            107   2022            2444  ...   
All          6.476845e+08            536  10246           12791  ...   

                                                                  PREM_AMT  \
CONSTR_MTRL modified firse resistive non-combustible    All fire resistive   
ST  INC_YR                                                                   
19  2014                          14             573   6825       399727.0   
    2015                          13             550   6731       495395.0   
    2016                          14             536   6620       658596.0   
    2017                          15             553   6606       644850.0   
    2018                          18             548   6501       688464.0   
All                               74            2760  33283      2887032.0   

                                                                 \
CONSTR_MTRL       frame joisted masonry masonry non-combustible   
ST  INC_YR                                                        
19  2014      5426771.0       6309872.0               4015886.0   
    2015      6124989.0       6599896.0               4217716.0   
    2016      7168403.0       6871030.0               4399870.0   
    2017      7908189.0       7092819.0               4530342.0   
    2018      8495693.0       7462963.0               4663902.0   
All          35124045.0      34336580.0              21827716.0   

                                                                   
CONSTR_MTRL modified firse resistive non-combustible          All  
ST  INC_YR                                                         
19  2014                     36918.0       1356293.0   17545467.0  
    2015                     51766.0       1423928.0   18913690.0  
    2016                     61838.0       1411952.0   20571689.0  
    2017                     83263.0       1557773.0   21817236.0  
    2018                    123527.0       1621762.0   23056311.0  
All                         357312.0       7371708.0  101904393.0  

[6 rows x 21 columns]

### Extracting Data to CSV
<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html">Pandas Write Data Frame to CSV</a>

In [ ]:
pivot2.to_csv('extract_pivot.csv', sep=',', na_rep='NA')

## Cleanup

In [ ]:
sf.close_connect()  # closes the Snowflake connection

## References
- [Greenplum SQL vs Snowflake SQL](https://epm.verisk.com/confluence/display/ISUIAE/GPDB+SQL+vs+SFDB+SQL)
- [Actuarial Routines Documentation](https://epm.verisk.com/confluence/pages/viewpage.action?pageId=177871408)
- [Pandas Write Data Frame to CSV](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html)
- [Pandas DataFrame Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html)